In [35]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, r2_score
import findspark
findspark.init()
from pyspark.sql import SparkSession
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [2]:
spark = SparkSession.builder.appName("Read CSV").getOrCreate()


24/03/06 19:19:56 WARN Utils: Your hostname, Aarons-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
24/03/06 19:19:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 19:19:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file_path = "Credit_Score_Data/Credit.csv"


In [4]:
df = spark.read.csv(file_path, header=True, inferSchema=True, sep=",")

df.show()

24/03/06 19:20:09 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-----------+-----+---------------+----+------------+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+------------------+---------------------+-------------------+-----------------------+--------------------+------------------+------------+
|  ID|Customer_ID|Month|           Name| Age|         SSN|Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|   Monthly_Balance|Credit_Score|
+----+-----------+-----+---------------+----+------------+----------+-----

In [5]:
credit_df= df.toPandas()

credit_df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,5634,3392,1,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,26.822620,265.0,No,49.574949,21.46538,High_spent_Small_value_payments,312.494089,Good
1,5635,3392,2,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,31.944960,266.0,No,49.574949,21.46538,Low_spent_Large_value_payments,284.629162,Good
2,5636,3392,3,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,28.609352,267.0,No,49.574949,21.46538,Low_spent_Medium_value_payments,331.209863,Good
3,5637,3392,4,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,31.377862,268.0,No,49.574949,21.46538,Low_spent_Small_value_payments,223.451310,Good
4,5638,3392,5,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,...,Good,809.98,24.797347,269.0,No,49.574949,21.46538,High_spent_Medium_value_payments,341.489231,Good


In [6]:
spark.stop()

In [7]:
credit_df['Payment_of_Min_Amount'] = credit_df['Payment_of_Min_Amount']. apply(lambda x:1 if x== 'yes' else 0)

In [8]:
non_numeric = ['Name','Occupation','Type_of_Loan','Credit_Mix','Payment_Behaviour','Credit_Score']

In [9]:
df_numeric = credit_df.drop(columns= non_numeric)

In [10]:
X = df_numeric
y = pd.get_dummies(credit_df["Credit_Score"])


In [11]:
df_categorical= credit_df[non_numeric]

In [12]:
df_categorical = pd.get_dummies(df_categorical)

In [13]:
X= pd.concat([X, df_categorical], axis=1)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

In [31]:
X_test.info()


<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 75721 to 42410
Columns: 16438 entries, ID to Credit_Score_Standard
dtypes: bool(16416), float64(18), int32(3), int64(1)
memory usage: 316.4 MB


In [17]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80000 entries, 75220 to 15795
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Good      80000 non-null  bool 
 1   Poor      80000 non-null  bool 
 2   Standard  80000 non-null  bool 
dtypes: bool(3)
memory usage: 859.4 KB


In [18]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=25, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/25
2000/2000 [==============================] - 9s 4ms/step - loss: 3.8964 - accuracy: 0.5953 - val_loss: 3.9942 - val_accuracy: 0.5920
Epoch 2/25
2000/2000 [==============================] - 5s 2ms/step - loss: 3.9546 - accuracy: 0.5842 - val_loss: 3.8154 - val_accuracy: 0.5967
Epoch 3/25
2000/2000 [==============================] - 5s 2ms/step - loss: 3.7617 - accuracy: 0.6121 - val_loss: 4.7697 - val_accuracy: 0.6615
Epoch 4/25
2000/2000 [==============================] - 5s 3ms/step - loss: 3.6446 - accuracy: 0.6365 - val_loss: 3.3473 - val_accuracy: 0.6499
Epoch 5/25
2000/2000 [==============================] - 5s 3ms/step - loss: 3.2972 - accuracy: 0.6366 - val_loss: 3.2671 - val_accuracy: 0.6553
Epoch 6/25
2000/2000 [==============================] - 5s 3ms/step - loss: 3.1498 - accuracy: 0.6609 - val_loss: 3.2354 - val_accuracy: 0.6643
Epoch 7/25
2000/2000 [==============================] - 5s 3ms/step - loss: 3.1235 - accuracy: 0.6519 - val_loss: 3.2240 - val_accuracy:

In [37]:
# Evaluate the model
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test)
print("Accuracy = ", model_accuracy)

625/625 [==============================] - 1s 1ms/step - loss: 0.6405 - accuracy: 0.6666
Accuracy =  0.6665639877319336


In [32]:
# Convert X_test to float
X_test = X_test.astype(float)

In [33]:
# Make predictions
y_pred = model.predict(X_test)

625/625 [==============================] - 1s 1ms/step


In [36]:
accuracy = accuracy_score(y_test, y_pred)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous targets